AER 850 Machine Learning Project
Hussein Hamie 500876254
Mishran Haque 500896460

**Turbofan Engine Remaining Useful Life Prediction**
**DATASET:** NASA Turbofan Jet Engine Data Set

Data sets consists of multiple multivariate time series. Each data set is further divided into training and test subsets. Each time series is from a different engine i.e., the data can be considered to be from a fleet of engines of the same type. Each engine starts with different degrees of initial wear and manufacturing variation which is unknown to the user. This wear and variation is considered normal, i.e., it is not considered a fault condition. There are three operational settings that have a substantial effect on engine performance. These settings are also included in the data. The data is contaminated with sensor noise.

The engine is operating normally at the start of each time series, and develops a fault at some point during the series. In the training set, the fault grows in magnitude until system failure. In the test set, the time series ends some time prior to system failure. The objective of the competition is to predict the number of remaining operational cycles before failure in the test set, i.e., the number of operational cycles after the last cycle that the engine will continue to operate. Also provided a vector of true Remaining Useful Life (RUL) values for the test data.

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:
1) unit number
2) time, in cycles
3) operational setting 1
4) operational setting 2
5) operational setting 3
6) sensor measurement 1
7) sensor measurement 2
...
26) sensor measurement 26

Data Set: FD001
Train trjectories: 100
Test trajectories: 100
Conditions: ONE (Sea Level)
Fault Modes: ONE (HPC Degradation)

Data Set: FD002
Train trjectories: 260
Test trajectories: 259
Conditions: SIX
Fault Modes: ONE (HPC Degradation)

Data Set: FD003
Train trjectories: 100
Test trajectories: 100
Conditions: ONE (Sea Level)
Fault Modes: TWO (HPC Degradation, Fan Degradation)

Data Set: FD004
Train trjectories: 248
Test trajectories: 249
Conditions: SIX
Fault Modes: TWO (HPC Degradation, Fan Degradation)

Reference: A. Saxena, K. Goebel, D. Simon, and N. Eklund, ‘Damage Propagation Modeling for Aircraft Engine Run-to-Failure Simulation’, in the Proceedings of the 1st International Conference on Prognostics and Health Management (PHM08), Denver CO, Oct 2008.

In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
import sklearn
from sklearn.metrics import mean_squared_error, r2_score
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random
import warnings
#warnings.filterwarnings('ignore')

In [146]:
# define filepath to read data
dir_path = './CMaps/'
# define column names for easy indexing
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = sensor_names=[ "(Total temperature at fan inlet) (◦R)",
                              "(Total temperature at LPC outlet) (◦R)",
                              "(Total temperature at HPC outlet) (◦R)",
                              "(Total temperature at LPT outlet) (◦R)",
                              "(Pressure at fan inlet) (psia)",
                              "(Total pressure in bypass-duct) (psia)",
                              "(Total pressure at HPC outlet) (psia)",
                              "(Physical fan speed) (rpm)",
                              "(Physical core speed) (rpm)",
                              "(Engine pressure ratio(P50/P2)",
                              "(Static pressure at HPC outlet) (psia)",
                              "(Ratio of fuel flow to Ps30) (pps/psia)",
                              "(Corrected fan speed) (rpm)",
                              "(Corrected core speed) (rpm)",
                              "(Bypass Ratio) ",
                              "(Burner fuel-air ratio)",
                              "(Bleed Enthalpy)",
                              "(Demanded fan speed)",
                              "(Demanded corrected fan speed)",
                              "(HPT coolant bleed (lbm/s))",
                              "(LPT coolant bleed (lbm/s))" ]
col_names = index_names + setting_names + sensor_names

In [147]:
#Import datasets and put them in a pandas dataset
def get_data(file):
    return pd.read_csv(file, sep='\s+',header=None,index_col=False,names=col_names)
FD001_train = get_data('CMaps/train_FD001.txt')
FD001_test = get_data('CMaps/test_FD001.txt')
FD002_train = get_data('CMaps/train_FD002.txt')
FD002_test = get_data('CMaps/test_FD002.txt')
FD003_train = get_data('CMaps/train_FD003.txt')
FD003_test = get_data('CMaps/test_FD003.txt')
FD004_train = get_data('CMaps/train_FD004.txt')
FD004_test = get_data('CMaps/test_FD004.txt')

In [148]:
FD001_train.head()

,unit_number,time_cycles,setting_1,setting_2,setting_3,(Total temperature at fan inlet) (◦R),(Total temperature at LPC outlet) (◦R),(Total temperature at HPC outlet) (◦R),(Total temperature at LPT outlet) (◦R),(Pressure at fan inlet) (psia),...,(Ratio of fuel flow to Ps30) (pps/psia),(Corrected fan speed) (rpm),(Corrected core speed) (rpm),(Bypass Ratio),(Burner fuel-air ratio),(Bleed Enthalpy),(Demanded fan speed),(Demanded corrected fan speed),(HPT coolant bleed (lbm/s)),(LPT coolant bleed (lbm/s))
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [149]:
FD001_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20631 entries, 0 to 20630
Data columns (total 26 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   unit_number                              20631 non-null  int64  
 1   time_cycles                              20631 non-null  int64  
 2   setting_1                                20631 non-null  float64
 3   setting_2                                20631 non-null  float64
 4   setting_3                                20631 non-null  float64
 5   (Total temperature at fan inlet) (◦R)    20631 non-null  float64
 6   (Total temperature at LPC outlet) (◦R)   20631 non-null  float64
 7   (Total temperature at HPC outlet) (◦R)   20631 non-null  float64
 8   (Total temperature at LPT outlet) (◦R)   20631 non-null  float64
 9   (Pressure at fan inlet) (psia)           20631 non-null  float64
 10  (Total pressure in bypass-duct) (psia)   20631

In [150]:
FD001_train.describe(include='all')

,unit_number,time_cycles,setting_1,setting_2,setting_3,(Total temperature at fan inlet) (◦R),(Total temperature at LPC outlet) (◦R),(Total temperature at HPC outlet) (◦R),(Total temperature at LPT outlet) (◦R),(Pressure at fan inlet) (psia),...,(Ratio of fuel flow to Ps30) (pps/psia),(Corrected fan speed) (rpm),(Corrected core speed) (rpm),(Bypass Ratio),(Burner fuel-air ratio),(Bleed Enthalpy),(Demanded fan speed),(Demanded corrected fan speed),(HPT coolant bleed (lbm/s)),(LPT coolant bleed (lbm/s))
count,20631.000000,20631.000000,20631.000000,20631.000000,20631.0,2.063100e+04,20631.000000,20631.000000,20631.000000,2.063100e+04,...,20631.000000,20631.000000,20631.000000,20631.000000,2.063100e+04,20631.000000,20631.0,20631.0,20631.000000,20631.000000
mean,51.506568,108.807862,-0.000009,0.000002,100.0,5.186700e+02,642.680934,1590.523119,1408.933782,1.462000e+01,...,521.413470,2388.096152,8143.752722,8.442146,3.000000e-02,393.210654,2388.0,100.0,38.816271,23.289705
std,29.227633,68.880990,0.002187,0.000293,0.0,6.537152e-11,0.500053,6.131150,9.000605,3.394700e-12,...,0.737553,0.071919,19.076176,0.037505,1.556432e-14,1.548763,0.0,0.0,0.180746,0.108251
min,1.000000,1.000000,-0.008700,-0.000600,100.0,5.186700e+02,641.210000,1571.040000,1382.250000,1.462000e+01,...,518.690000,2387.880000,8099.940000,8.324900,3.000000e-02,388.000000,2388.0,100.0,38.140000,22.894200
25%,26.000000,52.000000,-0.001500,-0.000200,100.0,5.186700e+02,642.325000,1586.260000,1402.360000,1.462000e+01,...,520.960000,2388.040000,8133.245000,8.414900,3.000000e-02,392.000000,2388.0,100.0,38.700000,23.221800
50%,52.000000,104.000000,0.000000,0.000000,100.0,5.186700e+02,642.640000,1590.100000,1408.040000,1.462000e+01,...,521.480000,2388.090000,8140.540000,8.438900,3.000000e-02,393.000000,2388.0,100.0,38.830000,23.297900
75%,77.000000,156.000000,0.001500,0.000300,100.0,5.186700e+02,643.000000,1594.380000,1414.555000,1.462000e+01,...,521.950000,2388.140000,8148.310000,8.465600,3.000000e-02,394.000000,2388.0,100.0,38.950000,23.366800
max,100.000000,362.000000,0.008700,0.000600,100.0,5.186700e+02,644.530000,1616.910000,1441.490000,1.462000e+01,...,523.380000,2388.560000,8293.720000,8.584800,3.000000e-02,400.000000,2388.0,100.0,39.430000,23.618400


In [151]:
#Finding missing/incorrect data
FD001_train.isnull().sum()

unit_number                                0
time_cycles                                0
setting_1                                  0
setting_2                                  0
setting_3                                  0
(Total temperature at fan inlet) (◦R)      0
(Total temperature at LPC outlet) (◦R)     0
(Total temperature at HPC outlet) (◦R)     0
(Total temperature at LPT outlet) (◦R)     0
(Pressure at fan inlet) (psia)             0
(Total pressure in bypass-duct) (psia)     0
(Total pressure at HPC outlet) (psia)      0
(Physical fan speed) (rpm)                 0
(Physical core speed) (rpm)                0
(Engine pressure ratio(P50/P2)             0
(Static pressure at HPC outlet) (psia)     0
(Ratio of fuel flow to Ps30) (pps/psia)    0
(Corrected fan speed) (rpm)                0
(Corrected core speed) (rpm)               0
(Bypass Ratio)                             0
(Burner fuel-air ratio)                    0
(Bleed Enthalpy)                           0
(Demanded 

In [152]:
#Creating a data profiling report to get understand the data better
from ydata_profiling import ProfileReport
#data_report = ProfileReport(FD001_train)
#data_report.to_file("TrainDataset_Profile.html")

In [153]:
#Dropping columns that do not contain necessary features (unit_number and operational settings)
drop_cols = index_names + setting_names
FD001_train = FD001_train.drop(columns = drop_cols)

#Dropping feature columns with constant values / values that do not give meaningful information
unique = FD001_train.nunique()
const_cols = unique[unique == 1].index.tolist()
FD001_train = FD001_train.drop(columns = const_cols)
FD001_train = FD001_train.drop(columns = "(Total pressure in bypass-duct) (psia)")

In [154]:
#Normalizing the data using min-max scaling
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
FD001_train[0:] = minmax_scaler.fit_transform(FD001_train[0:])
FD001_train


,(Total temperature at LPC outlet) (◦R),(Total temperature at HPC outlet) (◦R),(Total temperature at LPT outlet) (◦R),(Total pressure at HPC outlet) (psia),(Physical fan speed) (rpm),(Physical core speed) (rpm),(Static pressure at HPC outlet) (psia),(Ratio of fuel flow to Ps30) (pps/psia),(Corrected fan speed) (rpm),(Corrected core speed) (rpm),(Bypass Ratio),(Bleed Enthalpy),(HPT coolant bleed (lbm/s)),(LPT coolant bleed (lbm/s))
0,0.183735,0.406802,0.309757,0.726248,0.242424,0.109755,0.369048,0.633262,0.205882,0.199608,0.363986,0.333333,0.713178,0.724662
1,0.283133,0.453019,0.352633,0.628019,0.212121,0.100242,0.380952,0.765458,0.279412,0.162813,0.411312,0.333333,0.666667,0.731014
2,0.343373,0.369523,0.370527,0.710145,0.272727,0.140043,0.250000,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375
3,0.343373,0.256159,0.331195,0.740741,0.318182,0.124518,0.166667,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386
4,0.349398,0.257467,0.404625,0.668277,0.242424,0.149960,0.255952,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,0.686747,0.587312,0.782917,0.254428,0.439394,0.196491,0.726190,0.170576,0.558824,0.194344,0.656791,0.750000,0.271318,0.109500
20627,0.701807,0.729453,0.866475,0.162641,0.500000,0.194651,0.708333,0.211087,0.500000,0.188668,0.727203,0.583333,0.124031,0.366197
20628,0.665663,0.684979,0.775321,0.175523,0.515152,0.198196,0.738095,0.281450,0.529412,0.212148,0.922278,0.833333,0.232558,0.053991
20629,0.608434,0.746021,0.747468,0.133655,0.530303,0.233285,0.916667,0.208955,0.514706,0.203065,0.823394,0.583333,0.116279,0.234466


In [155]:
#Creating a data profiling report after processing
#data_report = ProfileReport(FD001_train)
#data_report.to_file("TrainDataset_Profile_PostProcess.html")

In [156]:
#Creating a function returns the pre-processed data from the dataset (drop unnecessary features and normalize).
#This can be used for subsequent datasets without repeating the same code for each set

def pre_process(dataset):
    dataset = dataset.drop(columns = drop_cols + const_cols + ["(Total pressure in bypass-duct) (psia)"])
    dataset[0:] = minmax_scaler.fit_transform(dataset[0:])
    return dataset

FD002_train = pre_process(FD002_train)
FD003_train = pre_process(FD003_train)
FD004_train = pre_process(FD004_train)
FD001_test = pre_process(FD001_test)
FD002_test = pre_process(FD002_test)
FD003_test = pre_process(FD003_test)
FD004_test = pre_process(FD004_test)